# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import sys
import time
sys.path.append("../")
# Import API key
from api_keys import g_key

# Store Part I results into DataFrame

#Load the csv exported in Part I to a DataFrame
#Temperature (F) vs. Latitude
#Humidity (%) vs. Latitude
#Cloudiness (%) vs. Latitude
#Wind Speed (mph) vs. Latitude

In [3]:
#read csv data 
city_df = pd.read_csv("../output_data/cities.csv")
city_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


## Humidity Heatmap
- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.

In [9]:
#create configure gmaps 
gmaps.configure(api_key=g_key)
locations = city_df[['Lat', 'Lng']]
humidity = city_df['Humidity']
max_humidity = (city_df["Humidity"]).max()

In [30]:
# fig = gmaps.figure()
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [75]:
#narrow the data with the conditions and drop any null values
narrow_city = city_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Cloudiness"] == 0) &
                          (city_df["Humidity"]>= 50) & (city_df["Max Temp"]<= 80)].dropna()
narrow_city

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
11,11,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24
33,33,arman,0,RU,1558378760,64,59.70,150.17,31.56,5.30
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05
57,57,kolyvan,0,RU,1558378764,87,55.31,82.74,48.20,4.47
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
112,112,bredasdorp,0,ZA,1558378775,62,-34.53,20.04,55.00,8.05


### Hotel Map
- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [81]:
# add the hotel name in the columns
hotel_df = pd.DataFrame(narrow_city)
narrow_city["Hotel Name"] = ""
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
11,11,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17,
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26,
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24,
33,33,arman,0,RU,1558378760,64,59.70,150.17,31.56,5.30,
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05,


In [82]:
#Creating heatmap with labels and InfoWindow
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "lodging",
         "keyword" : "hotel",
         "radius" : 5000,
         "key":g_key}
#set variable in row 
for index, row in city_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    

    params["location"] = f"{lat},{lng}"
    
    print(f"The results for index{index}:{city_name}.")
    response = requests.get(url, params=params)
    response = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except(KeyError,IndexError):
        print("No Results found... skipppppp.")
        print("``````````````````````````````")


The results for index0:qingdao.
No Results found... skipppppp.
``````````````````````````````
The results for index1:qingdao.
The results for index2:qingdao.
The results for index3:qingdao.
No Results found... skipppppp.
``````````````````````````````
The results for index4:qingdao.
The results for index5:qingdao.
The results for index6:qingdao.
The results for index7:qingdao.
The results for index8:qingdao.
The results for index9:qingdao.
The results for index10:qingdao.
The results for index11:qingdao.
The results for index12:qingdao.
The results for index13:qingdao.
No Results found... skipppppp.
``````````````````````````````
The results for index14:qingdao.
The results for index15:qingdao.
The results for index16:qingdao.
The results for index17:qingdao.
The results for index18:qingdao.
The results for index19:qingdao.
The results for index20:qingdao.
The results for index21:qingdao.
The results for index22:qingdao.
The results for index23:qingdao.
The results for index24:qingdao.

The results for index181:qingdao.
The results for index182:qingdao.
No Results found... skipppppp.
``````````````````````````````
The results for index183:qingdao.
The results for index184:qingdao.
The results for index185:qingdao.
No Results found... skipppppp.
``````````````````````````````
The results for index186:qingdao.
No Results found... skipppppp.
``````````````````````````````
The results for index187:qingdao.
The results for index188:qingdao.
The results for index189:qingdao.
The results for index190:qingdao.
The results for index191:qingdao.
The results for index192:qingdao.
The results for index193:qingdao.
The results for index194:qingdao.
The results for index195:qingdao.
The results for index196:qingdao.
The results for index197:qingdao.
The results for index198:qingdao.
The results for index199:qingdao.
The results for index200:qingdao.
The results for index201:qingdao.
The results for index202:qingdao.
The results for index203:qingdao.
The results for index204:qingdao

No Results found... skipppppp.
``````````````````````````````
The results for index368:qingdao.
The results for index369:qingdao.
The results for index370:qingdao.
The results for index371:qingdao.
The results for index372:qingdao.
The results for index373:qingdao.
The results for index374:qingdao.
The results for index375:qingdao.
The results for index376:qingdao.
The results for index377:qingdao.
The results for index378:qingdao.
The results for index379:qingdao.
No Results found... skipppppp.
``````````````````````````````
The results for index380:qingdao.
The results for index381:qingdao.
The results for index382:qingdao.
The results for index383:qingdao.
The results for index384:qingdao.
No Results found... skipppppp.
``````````````````````````````
The results for index385:qingdao.
The results for index386:qingdao.
The results for index387:qingdao.
The results for index388:qingdao.
The results for index389:qingdao.
The results for index390:qingdao.
The results for index391:qingdao

In [92]:
#recheck if gmaps response for hotel found 
hotel_df = hotel_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Cloudiness"] == 0) &
                          (city_df["Humidity"]>= 50) & (city_df["Max Temp"]<= 80)].dropna()
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
11,11.0,castro,0.0,CL,1.558379e+09,70.0,-42.48,-73.76,48.20,9.17,Enjoy Chiloé. Hotel De La Isla
13,13.0,saskylakh,0.0,RU,1.558379e+09,91.0,71.97,114.09,4.92,2.26,
31,31.0,qingdao,0.0,CN,1.558379e+09,54.0,36.09,120.35,59.00,2.24,Shangri-La Hotel Qingdao
33,33.0,arman,0.0,RU,1.558379e+09,64.0,59.70,150.17,31.56,5.30,
37,37.0,varhaug,0.0,NO,1.558379e+09,63.0,58.61,5.65,64.99,8.05,
57,57.0,kolyvan,0.0,RU,1.558379e+09,87.0,55.31,82.74,48.20,4.47,
70,70.0,ponta do sol,0.0,BR,1.558379e+09,57.0,-20.63,-46.00,73.68,5.32,Escarpas Resort
84,84.0,emerald,0.0,AU,1.558379e+09,82.0,-23.53,148.16,60.80,6.93,Direct Hotels - Villas on Rivergum
88,88.0,cidreira,0.0,BR,1.558379e+09,66.0,-30.17,-50.22,76.56,5.50,Hotel Castelo
112,112.0,bredasdorp,0.0,ZA,1.558379e+09,62.0,-34.53,20.04,55.00,8.05,Victoria Hotel - Bredasdorp


In [93]:
#using template to add hotel in the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
fig1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=hotel_df['Humidity'], 
                                 max_intensity = max_humidity,dissipating=False,point_radius=2)
fig1.add_layer(heat_layer)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig1.add_layer(markers)

# Display Map
fig1

Figure(layout=FigureLayout(height='420px'))